In [ ]:
from EasyOCR.easyocr import *
import os
import cv2

# GPU 설정
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

def get_files(path):
    file_list = []

    files = [f for f in os.listdir(path) if not f.startswith('.')]
    files.sort()
    abspath = os.path.abspath(path)
    print(abspath)
    for file in files:
        file_path = os.path.join(abspath, file)
        file_list.append(file_path)

    return file_list, len(file_list)

reader = Reader(['ko'], gpu=True,
                model_storage_directory='model',
                user_network_directory='EasyOCR\\trainer\\config_files',
                recog_network='custom')


files, count = get_files('demo_image')  

for idx, file in enumerate(files):
    filename = os.path.basename(file)
    # 경로에 한글이 들어간다면 에러 발생 가능성이 높습니다.
    img = cv2.imread(r'./demo_image/plate.jpg')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    result = reader.recognize(gray)

    for (bbox, string, confidence) in result:   
        print("filename: '%s', confidence: %.4f, string: '%s'" % (filename, confidence, string))

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
c:\Users\다인\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\functional.py:505: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:4319.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
c:\Users\다인\EASYOCR-custom\EasyOCR\trainer\config_files\custom.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("inv_delta_C", torch.tensor(self._build_inv_delta_C(self.F, self.C)).float())
c:\Users\다인\EASYOCR-custom\EasyOCR\trainer\config_files\custom.py:124: UserWarning: To copy construct from a tensor, it is recommended to use so

c:\Users\다인\EASYOCR-custom\demo_image
filename: 'plate.jpg', confidence: 0.9996, string: '206호1213[s][s][s][s][s][s][s][s][s][s][s][s][s][s][s][s][s][s]'


c:\Users\다인\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
